<a href="https://colab.research.google.com/github/djdamie/Retrieval-based-Voice-Conversion-WebUI/blob/main/TF_Retrieval_based_Voice_Conversion_WebUI_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/RVC-Project/Retrieval-based-Voice-Conversion-WebUI/blob/main/Retrieval_based_Voice_Conversion_WebUI_v2.ipynb)

In [2]:
#@title 查看显卡
!nvidia-smi

Sun Jul  9 15:01:44 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   63C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
#@title 安装依赖
!apt-get -y install build-essential python3-dev ffmpeg
!pip3 install --upgrade setuptools wheel
!pip3 install --upgrade pip
!pip3 install faiss-cpu==1.7.2 fairseq gradio==3.14.0 ffmpeg ffmpeg-python praat-parselmouth pyworld numpy==1.23.5 numba==0.56.4 librosa==0.9.2

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3-dev is already the newest version (3.8.2-0ubuntu2).
build-essential is already the newest version (12.8ubuntu1.1).
ffmpeg is already the newest version (7:4.2.7-0ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 15 not upgraded.
  Using cached faiss_cpu-1.7.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (8.6 MB)
  Using cached fairseq-0.12.2.tar.gz (9.6 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached gradio-3.14.0-py3-none-any.whl (13.8 MB)
  Using cached ffmpeg-1.4.tar.gz (5.1 kB)
  Preparing metadata (setup.py) ... done
  Using cached ffmpeg_python-0.2.0-py3-none-any.whl (25 kB)
  Using cached praat_parselmouth-0.4.3-cp310-cp310-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (10.7 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
#@title 克隆仓库

!mkdir Retrieval-based-Voice-Conversion-WebUI
%cd /content/Retrieval-based-Voice-Conversion-WebUI
!git init
!git remote add origin https://github.com/RVC-Project/Retrieval-based-Voice-Conversion-WebUI.git
!git fetch origin cfd984812804ddc9247d65b14c82cd32e56c1133 --depth=1
!git reset --hard FETCH_HEAD

mkdir: cannot create directory ‘Retrieval-based-Voice-Conversion-WebUI’: File exists
/content/Retrieval-based-Voice-Conversion-WebUI
Reinitialized existing Git repository in /content/Retrieval-based-Voice-Conversion-WebUI/.git/
fatal: remote origin already exists.
remote: Total 0 (delta 0), reused 0 (delta 0), pack-reused 0
From https://github.com/RVC-Project/Retrieval-based-Voice-Conversion-WebUI
 * branch            cfd984812804ddc9247d65b14c82cd32e56c1133 -> FETCH_HEAD
HEAD is now at cfd9848 Format code (#330)


In [3]:
#@title 更新仓库（一般无需执行）
!git pull

remote: Enumerating objects: 629, done.
remote: Counting objects: 100% (629/629), done.
remote: Compressing objects: 100% (289/289), done.
remote: Total 584 (delta 390), reused 457 (delta 289), pack-reused 0
Receiving objects: 100% (584/584), 255.49 KiB | 1.33 MiB/s, done.
Resolving deltas: 100% (390/390), completed with 35 local objects.
From https://github.com/RVC-Project/Retrieval-based-Voice-Conversion-WebUI
 * [new branch]      create-pull-request/patch -> origin/create-pull-request/patch
 * [new branch]      main       -> origin/main
 * [new tag]         updated0528v2 -> updated0528v2
 * [new tag]         updated0618v2 -> updated0618v2
There is no tracking information for the current branch.
Please specify which branch you want to merge with.
See git-pull(1) for details.

    git pull <remote> <branch>

If you wish to set tracking information for this branch you can do so with:

    git branch --set-upstream-to=origin/<branch> master



In [4]:
#@title 安装aria2
!apt -y install -qq aria2

The following additional packages will be installed:
  libaria2-0 libc-ares2
The following NEW packages will be installed:
  aria2 libaria2-0 libc-ares2
0 upgraded, 3 newly installed, 0 to remove and 15 not upgraded.
Need to get 1,475 kB of archives.
After this operation, 5,959 kB of additional disk space will be used.
Selecting previously unselected package libc-ares2:amd64.
(Reading database ... 123105 files and directories currently installed.)
Preparing to unpack .../libc-ares2_1.15.0-1ubuntu0.3_amd64.deb ...
Unpacking libc-ares2:amd64 (1.15.0-1ubuntu0.3) ...
Selecting previously unselected package libaria2-0:amd64.
Preparing to unpack .../libaria2-0_1.35.0-1build1_amd64.deb ...
Unpacking libaria2-0:amd64 (1.35.0-1build1) ...
Selecting previously unselected package aria2.
Preparing to unpack .../aria2_1.35.0-1build1_amd64.deb ...
Unpacking aria2 (1.35.0-1build1) ...
Setting up libc-ares2:amd64 (1.15.0-1ubuntu0.3) ...
Setting up libaria2-0:amd64 (1.35.0-1build1) ...
Setting up aria2

In [5]:
#@title 下载底模

# v1
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/D32k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained -o D32k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/D40k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained -o D40k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/D48k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained -o D48k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/G32k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained -o G32k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/G40k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained -o G40k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/G48k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained -o G48k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/f0D32k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained -o f0D32k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/f0D40k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained -o f0D40k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/f0D48k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained -o f0D48k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/f0G32k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained -o f0G32k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/f0G40k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained -o f0G40k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/f0G48k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained -o f0G48k.pth

#v2
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained_v2/D32k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained_v2 -o D32k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained_v2/D40k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained_v2 -o D40k.pth
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained_v2/D48k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained_v2 -o D48k.pth
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained_v2/G32k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained_v2 -o G32k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained_v2/G40k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained_v2 -o G40k.pth
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained_v2/G48k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained_v2 -o G48k.pth
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained_v2/f0D32k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained_v2 -o f0D32k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained_v2/f0D40k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained_v2 -o f0D40k.pth
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained_v2/f0D48k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained_v2 -o f0D48k.pth
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained_v2/f0G32k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained_v2 -o f0G32k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained_v2/f0G40k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained_v2 -o f0G40k.pth
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained_v2/f0G48k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained_v2 -o f0G48k.pth


Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
973880|OK  |   137MiB/s|/content/Retrieval-based-Voice-Conversion-WebUI/pretrained/D32k.pth

Status Legend:
(OK):download completed.

Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
593c78|OK  |   100MiB/s|/content/Retrieval-based-Voice-Conversion-WebUI/pretrained/D40k.pth

Status Legend:
(OK):download completed.

Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
5efee3|OK  |   147MiB/s|/content/Retrieval-based-Voice-Conversion-WebUI/pretrained/D48k.pth

Status Legend:
(OK):download completed.

Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
4e0bf1|OK  |   191MiB/s|/content/Retrieval-based-Voice-Conversion-WebUI/p

In [6]:
#@title 下载人声分离模型
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/uvr5_weights/HP2-人声vocals+非人声instrumentals.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/uvr5_weights -o HP2-人声vocals+非人声instrumentals.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/uvr5_weights/HP5-主旋律人声vocals+其他instrumentals.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/uvr5_weights -o HP5-主旋律人声vocals+其他instrumentals.pth


Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
daf99a|OK  |   334MiB/s|/content/Retrieval-based-Voice-Conversion-WebUI/uvr5_weights/HP2-人声vocals+非人声instrumentals.pth

Status Legend:
(OK):download completed.

Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
f01354|OK  |   260MiB/s|/content/Retrieval-based-Voice-Conversion-WebUI/uvr5_weights/HP5-主旋律人声vocals+其他instrumentals.pth

Status Legend:
(OK):download completed.


In [7]:
#@title 下载hubert_base
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/hubert_base.pt -d /content/Retrieval-based-Voice-Conversion-WebUI -o hubert_base.pt


Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
3e0553|OK  |   200MiB/s|/content/Retrieval-based-Voice-Conversion-WebUI/hubert_base.pt

Status Legend:
(OK):download completed.


In [ ]:
#@title 挂载谷歌云盘

from google.colab import drive
drive.mount('/content/drive')

In [11]:
#@title 从谷歌云盘加载打包好的数据集到/content/dataset

#@markdown 数据集位置
DATASET = "/content/drive/MyDrive/dataset/lulu20230327_32k.zip"  #@param {type:"string"}

!mkdir -p /content/dataset
!unzip -d /content/dataset -B {DATASET}

unzip:  cannot find or open /content/drive/MyDrive/dataset/lulu20230327_32k.zip, /content/drive/MyDrive/dataset/lulu20230327_32k.zip.zip or /content/drive/MyDrive/dataset/lulu20230327_32k.zip.ZIP.


In [12]:
#@title 重命名数据集中的重名文件
!ls -a /content/dataset/
!rename 's/(\w+)\.(\w+)~(\d*)/$1_$3.$2/' /content/dataset/*.*~*

.  ..


In [ ]:
#@title 启动web
%cd /content/Retrieval-based-Voice-Conversion-WebUI
# %load_ext tensorboard
# %tensorboard --logdir /content/Retrieval-based-Voice-Conversion-WebUI/logs
!python3 infer-web.py --colab --pycmd python3

/content/Retrieval-based-Voice-Conversion-WebUI
Use Language: en_US
2023-07-09 16:53:07.597188: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-09 16:53:08.538515: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-07-09 16:53:10 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
Running on local URL:  http://127.0.0.1:7860

Setting up a public link... we have recently upgraded the way public links are generated. If you encounter any problems, please report the issue and downgrade to gradio version 3.13.0
.
Running on public URL: https://e32a406d-ff60-40ae.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!)

In [ ]:
#@title 手动将训练后的模型文件备份到谷歌云盘
#@markdown 需要自己查看logs文件夹下模型的文件名，手动修改下方命令末尾的文件名

#@markdown 模型名
MODELNAME = "lulu"  #@param {type:"string"}
#@markdown 模型epoch
MODELEPOCH = 9600  #@param {type:"integer"}

!cp /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/G_{MODELEPOCH}.pth /content/drive/MyDrive/{MODELNAME}_D_{MODELEPOCH}.pth
!cp /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/D_{MODELEPOCH}.pth /content/drive/MyDrive/{MODELNAME}_G_{MODELEPOCH}.pth
!cp /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/added_*.index /content/drive/MyDrive/
!cp /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/total_*.npy /content/drive/MyDrive/

!cp /content/Retrieval-based-Voice-Conversion-WebUI/weights/{MODELNAME}.pth /content/drive/MyDrive/{MODELNAME}{MODELEPOCH}.pth

In [ ]:
#@title 从谷歌云盘恢复pth
#@markdown 需要自己查看logs文件夹下模型的文件名，手动修改下方命令末尾的文件名

#@markdown 模型名
MODELNAME = "lulu"  #@param {type:"string"}
#@markdown 模型epoch
MODELEPOCH = 7500  #@param {type:"integer"}

!mkdir -p /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}

!cp /content/drive/MyDrive/{MODELNAME}_D_{MODELEPOCH}.pth /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/G_{MODELEPOCH}.pth
!cp /content/drive/MyDrive/{MODELNAME}_G_{MODELEPOCH}.pth /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/D_{MODELEPOCH}.pth
!cp /content/drive/MyDrive/*.index /content/
!cp /content/drive/MyDrive/*.npy /content/
!cp /content/drive/MyDrive/{MODELNAME}{MODELEPOCH}.pth /content/Retrieval-based-Voice-Conversion-WebUI/weights/{MODELNAME}.pth

In [ ]:
#@title 手动预处理（不推荐）
#@markdown 模型名
MODELNAME = "lulu"  #@param {type:"string"}
#@markdown 采样率
BITRATE = 48000  #@param {type:"integer"}
#@markdown 使用的进程数
THREADCOUNT = 8  #@param {type:"integer"}

!python3 trainset_preprocess_pipeline_print.py /content/dataset {BITRATE} {THREADCOUNT} logs/{MODELNAME} True


In [ ]:
#@title 手动提取特征（不推荐）
#@markdown 模型名
MODELNAME = "lulu"  #@param {type:"string"}
#@markdown 使用的进程数
THREADCOUNT = 8  #@param {type:"integer"}
#@markdown 音高提取算法
ALGO = "harvest"  #@param {type:"string"}

!python3 extract_f0_print.py logs/{MODELNAME} {THREADCOUNT} {ALGO}

!python3 extract_feature_print.py cpu 1 0 0 logs/{MODELNAME}


In [ ]:
#@title 手动训练（不推荐）
#@markdown 模型名
MODELNAME = "lulu"  #@param {type:"string"}
#@markdown 使用的GPU
USEGPU = "0"  #@param {type:"string"}
#@markdown 批大小
BATCHSIZE = 32  #@param {type:"integer"}
#@markdown 停止的epoch
MODELEPOCH = 3200  #@param {type:"integer"}
#@markdown 保存epoch间隔
EPOCHSAVE = 100  #@param {type:"integer"}
#@markdown 采样率
MODELSAMPLE = "48k"  #@param {type:"string"}
#@markdown 是否缓存训练集
CACHEDATA = 1  #@param {type:"integer"}
#@markdown 是否仅保存最新的ckpt文件
ONLYLATEST = 0  #@param {type:"integer"}

!python3 train_nsf_sim_cache_sid_load_pretrain.py -e lulu -sr {MODELSAMPLE} -f0 1 -bs {BATCHSIZE} -g {USEGPU} -te {MODELEPOCH} -se {EPOCHSAVE} -pg pretrained/f0G{MODELSAMPLE}.pth -pd pretrained/f0D{MODELSAMPLE}.pth -l {ONLYLATEST} -c {CACHEDATA}


In [ ]:
#@title 删除其它pth，只留选中的（慎点，仔细看代码）
#@markdown 模型名
MODELNAME = "lulu"  #@param {type:"string"}
#@markdown 选中模型epoch
MODELEPOCH = 9600  #@param {type:"integer"}

!echo "备份选中的模型。。。"
!cp /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/G_{MODELEPOCH}.pth /content/{MODELNAME}_D_{MODELEPOCH}.pth
!cp /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/D_{MODELEPOCH}.pth /content/{MODELNAME}_G_{MODELEPOCH}.pth

!echo "正在删除。。。"
!ls /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}
!rm /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/*.pth

!echo "恢复选中的模型。。。"
!mv /content/{MODELNAME}_D_{MODELEPOCH}.pth /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/G_{MODELEPOCH}.pth
!mv /content/{MODELNAME}_G_{MODELEPOCH}.pth /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/D_{MODELEPOCH}.pth

!echo "删除完成"
!ls /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}

In [ ]:
#@title 清除项目下所有文件，只留选中的模型（慎点，仔细看代码）
#@markdown 模型名
MODELNAME = "lulu"  #@param {type:"string"}
#@markdown 选中模型epoch
MODELEPOCH = 9600  #@param {type:"integer"}

!echo "备份选中的模型。。。"
!cp /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/G_{MODELEPOCH}.pth /content/{MODELNAME}_D_{MODELEPOCH}.pth
!cp /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/D_{MODELEPOCH}.pth /content/{MODELNAME}_G_{MODELEPOCH}.pth

!echo "正在删除。。。"
!ls /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}
!rm -rf /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/*

!echo "恢复选中的模型。。。"
!mv /content/{MODELNAME}_D_{MODELEPOCH}.pth /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/G_{MODELEPOCH}.pth
!mv /content/{MODELNAME}_G_{MODELEPOCH}.pth /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/D_{MODELEPOCH}.pth

!echo "删除完成"
!ls /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}